In [1]:
from typing import Tuple

import itertools

import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'usa'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> Prophet:
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(**params).fit(df)
    return optimal_model

In [8]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, param_grid)

22:57:13 - cmdstanpy - INFO - Chain [1] start processing
22:57:13 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] done processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [1] start processing
22:57:14 - cmdstanpy - INFO - Chain [

In [9]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [10]:
df_predicted, rmse = test_model(df_test, model)

In [11]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-04-01,4.364488,3.215400,5.815320,4.364488,4.364488,0.146732,0.146732,0.146732,0.146732,0.146732,0.146732,0.0,0.0,0.0,4.511220
1,2022-05-01,4.341964,3.156404,5.784665,4.341964,4.341964,0.122117,0.122117,0.122117,0.122117,0.122117,0.122117,0.0,0.0,0.0,4.464081
2,2022-06-01,4.318690,3.027829,5.774673,4.318690,4.318690,0.111114,0.111114,0.111114,0.111114,0.111114,0.111114,0.0,0.0,0.0,4.429804
3,2022-07-01,4.296166,3.175841,5.556186,4.296166,4.296166,0.071797,0.071797,0.071797,0.071797,0.071797,0.071797,0.0,0.0,0.0,4.367963
4,2022-08-01,4.272892,3.050602,5.611893,4.272892,4.272892,0.042564,0.042564,0.042564,0.042564,0.042564,0.042564,0.0,0.0,0.0,4.315455
5,2022-09-01,4.249617,2.918859,5.524038,4.249617,4.249617,0.021653,0.021653,0.021653,0.021653,0.021653,0.021653,0.0,0.0,0.0,4.271270
6,2022-10-01,4.227093,2.919506,5.489922,4.227093,4.235393,-0.001354,-0.001354,-0.001354,-0.001354,-0.001354,-0.001354,0.0,0.0,0.0,4.225739
7,2022-11-01,4.203819,2.851191,5.598160,4.195002,4.222142,0.009517,0.009517,0.009517,0.009517,0.009517,0.009517,0.0,0.0,0.0,4.213336
8,2022-12-01,4.181295,2.908973,5.468392,4.160725,4.215875,-0.014442,-0.014442,-0.014442,-0.014442,-0.014442,-0.014442,0.0,0.0,0.0,4.166853
9,2023-01-01,4.158020,2.846509,5.496773,4.123662,4.205730,-0.015041,-0.015041,-0.015041,-0.015041,-0.015041,-0.015041,0.0,0.0,0.0,4.142979


In [12]:
rmse

0.7370049733556286

In [13]:
with open(f'{country}_prophet_lockdowns_optimized_model.json', 'w') as f:
    f.write(model_to_json(model))

In [14]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [15]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [16]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,4.113724,3.004349,5.582627,4.113724,4.113724,0.177944,0.177944,0.177944,0.177944,0.177944,0.177944,0.0,0.0,0.0,4.291667
1,2023-04-01,4.090449,2.933745,5.444391,4.090449,4.090449,0.108196,0.108196,0.108196,0.108196,0.108196,0.108196,0.0,0.0,0.0,4.198645
2,2023-05-01,4.067925,2.856417,5.479863,4.067925,4.067925,0.098752,0.098752,0.098752,0.098752,0.098752,0.098752,0.0,0.0,0.0,4.166677
